## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barkhan,PK,29.8977,69.5256,91.71,6,0,8.16,clear sky
1,1,Avarua,CK,-21.2078,-159.7750,75.25,94,75,10.36,shower rain
2,2,Clyde River,CA,70.4692,-68.5914,16.03,92,100,18.41,light snow
3,3,Bengkulu,ID,-3.8004,102.2655,86.05,67,74,17.27,broken clouds
4,4,Rikitea,PF,-23.1203,-134.9692,76.87,82,9,19.73,light rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Avarua,CK,-21.2078,-159.7750,75.25,94,75,10.36,shower rain
3,3,Bengkulu,ID,-3.8004,102.2655,86.05,67,74,17.27,broken clouds
4,4,Rikitea,PF,-23.1203,-134.9692,76.87,82,9,19.73,light rain
5,5,Butaritari,KI,3.0707,172.7902,80.98,79,67,9.69,broken clouds
6,6,Sabang,ID,5.8933,95.3214,84.15,74,100,21.07,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             182
City                   182
Country                182
Lat                    182
Lng                    182
Max Temp               182
Humidity               182
Cloudiness             182
Wind Speed             182
Current Description    182
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,75.25,shower rain,-21.2078,-159.7750,
3,Bengkulu,ID,86.05,broken clouds,-3.8004,102.2655,
4,Rikitea,PF,76.87,light rain,-23.1203,-134.9692,
5,Butaritari,KI,80.98,broken clouds,3.0707,172.7902,
6,Sabang,ID,84.15,light rain,5.8933,95.3214,
7,La Cruz,MX,77.27,broken clouds,23.9167,-106.9000,
8,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,
9,Bud,US,75.04,clear sky,39.4470,-86.1758,
11,Victoria,HK,79.95,heavy intensity rain,22.2855,114.1577,
12,Mahebourg,MU,76.24,scattered clouds,-20.4081,57.7000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   182
Country                182
Max Temp               182
Current Description    182
Lat                    182
Lng                    182
Hotel Name             182
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
clean_hotel_df=hotel_df
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))